In [1]:
import os
import json
import numpy as np
import torch
import result
import pandas as pd
from rnn_model import LSTMNet, GRUNet
from utils import DotDict, Logger, rmse, rmse_tensor, rmse_np, boolean_string, get_dir, get_time, next_dir, model_dir, get_model
from get_dataset import get_time_data, get_stnn_data
from stnn import *
import matplotlib.pyplot as plt
from matplotlib import axis
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
%matplotlib inline

%load_ext autoreload
%autoreload 2
%reload_ext autoreload


# Batch Size and its compact 

## Batch Jar

In [4]:
best_jar_folder = os.path.abspath(os.path.join(os.getcwd(), '..', 'output_result_batch', 'batch_jar'))
jar_result = result.Printer(best_jar_folder)
pd.options.display.max_rows = None
# a = jar_result.get_df(col=['train_loss', 'test_loss', 'mode']).sort_values('test_loss', ascending=True)
jar_df = jar_result.get_df(col=['minrmse','data_normalize', 'batch_size', 'nz', 'nlayers', 'activation', 'nhid', 'time'], increase=False, required_list='classical').sort_values('time')
result.fliter_row(jar_df, "classical")

,minrmse,data_normalize,batch_size,nz,nlayers,activation,nhid,time
classical-stnn_07-28-15-26-44_7811,419.793,d,900,40,3,sigmoid,100,0:03:40.232568
classical-stnn_07-28-15-26-39_6441,419.793,d,700,40,3,sigmoid,100,0:03:56.282563
classical-stnn_07-28-15-26-38_2090,419.793,d,500,40,3,sigmoid,100,0:03:57.160261
classical-stnn_07-28-15-26-35_0441,407.212,d,300,40,3,sigmoid,100,0:05:14.369444
classical-stnn_07-28-15-26-31_4980,347.065,d,100,40,3,sigmoid,100,0:07:30.789629
classical-stnn_07-28-15-23-54_2101,237.278,d,10,40,3,sigmoid,100,0:58:49.140525


In [8]:
best_jar_folder = os.path.abspath(os.path.join(os.getcwd(), '..', 'output_result_batch', 'batch_jar'))
jar_result = result.Printer(best_jar_folder)
pd.options.display.max_rows = None
# a = jar_result.get_df(col=['train_loss', 'test_loss', 'mode']).sort_values('test_loss', ascending=True)
jar_df = jar_result.get_df(col=['minrmse','data_normalize', 'batch_size', 'nz', 'nlayers', 'activation', 'nhid', 'time'], increase=False, required_list='classical').sort_values('time')
result.fliter_row(jar_df, "v3")

,minrmse,data_normalize,batch_size,nz,nlayers,activation,nhid,time
v3-stnn_07-28-15-26-44_4415,218.507,x,900,10,2,tanh,100,0:03:34.433705
v3-stnn_07-28-15-26-39_8812,218.507,x,700,10,2,tanh,100,0:03:45.358356
v3-stnn_07-28-15-26-38_4611,218.507,x,500,10,2,tanh,100,0:03:46.104835
v3-stnn_07-28-15-26-35_6033,210.84,x,300,10,2,tanh,100,0:04:26.114015
v3-stnn_07-28-15-26-31_4247,205.822,x,100,10,2,tanh,100,0:05:32.361254
v3-stnn_07-28-15-23-54_7736,173.006,x,10,10,2,tanh,100,0:53:41.413550


In [9]:
best_jar_folder = os.path.abspath(os.path.join(os.getcwd(), '..', 'output_result_batch', 'batch_jar'))
jar_result = result.Printer(best_jar_folder)
pd.options.display.max_rows = None
# a = jar_result.get_df(col=['train_loss', 'test_loss', 'mode']).sort_values('test_loss', ascending=True)
jar_df = jar_result.get_df(col=['minrmse','data_normalize', 'batch_size', 'nz', 'nlayers', 'activation', 'nhid', 'time'], increase=False, required_list='classical').sort_values('time')
result.fliter_row(jar_df, "ori")

,minrmse,data_normalize,batch_size,nz,nlayers,activation,nhid,time
ori-stnn_07-28-15-26-44_3996,206.069,x,900,10,2,tanh,100,0:02:00.065429
ori-stnn_07-28-15-26-39_4837,206.069,x,700,10,2,tanh,100,0:02:10.823770
ori-stnn_07-28-15-26-38_1666,206.069,x,500,10,2,tanh,100,0:02:12.127736
ori-stnn_07-28-15-26-35_8277,189.401,x,300,10,2,tanh,100,0:03:03.285891
ori-stnn_07-28-15-26-31_4401,171.489,x,100,10,2,tanh,100,0:04:12.802136
ori-stnn_07-28-15-23-54_2150,109.232,x,10,10,2,tanh,100,0:39:53.185010


# Batch Feb

In [13]:
best_feb_folder = os.path.abspath(os.path.join(os.getcwd(), '..', 'output_result_batch', 'batch_feb'))
feb_result = result.Printer(best_feb_folder)
pd.options.display.max_rows = None
# a = jar_result.get_df(col=['train_loss', 'test_loss', 'mode']).sort_values('test_loss', ascending=True)
feb_df = feb_result.get_df(col=['minrmse','data_normalize', 'batch_size', 'nz', 'nlayers', 'activation', 'nhid', 'time'], increase=False, required_list='classical').sort_values('time')
result.fliter_row(feb_df, "classical")

,minrmse,data_normalize,batch_size,nz,nlayers,activation,nhid,time
classical-stnn_07-28-15-46-44_2546,71.8239,d,900,10,3,sigmoid,50,0:05:18.598914
classical-stnn_07-28-15-46-40_7729,596.885,d,500,10,3,sigmoid,50,0:05:28.898488
classical-stnn_07-28-15-46-40_9952,86.6205,d,700,10,3,sigmoid,50,0:07:15.476217
classical-stnn_07-28-15-46-40_4029,405.668,d,300,10,3,sigmoid,50,0:08:25.824487
classical-stnn_07-28-15-46-40_0797,550.115,d,100,10,3,sigmoid,50,0:14:55.654537
classical-stnn_07-28-15-43-10_0829,64.808,d,10,10,3,sigmoid,50,1:00:46.572241


In [14]:
best_feb_folder = os.path.abspath(os.path.join(os.getcwd(), '..', 'output_result_batch', 'batch_feb'))
feb_result = result.Printer(best_feb_folder)
pd.options.display.max_rows = None
# a = jar_result.get_df(col=['train_loss', 'test_loss', 'mode']).sort_values('test_loss', ascending=True)
feb_df = feb_result.get_df(col=['minrmse','data_normalize', 'batch_size', 'nz', 'nlayers', 'activation', 'nhid', 'time'], increase=False, required_list='classical').sort_values('time')
result.fliter_row(feb_df, "v3")

,minrmse,data_normalize,batch_size,nz,nlayers,activation,nhid,time
v3-stnn_07-28-15-46-44_1240,73.3028,d,900,10,2,sigmoid,100,0:05:34.316718
v3-stnn_07-28-15-46-40_5862,94.1115,d,700,10,2,sigmoid,100,0:06:09.405129
v3-stnn_07-28-15-46-40_9363,69.3639,d,500,10,2,sigmoid,100,0:08:04.225752
v3-stnn_07-28-15-46-40_3243,57.1387,d,300,10,2,sigmoid,100,0:08:48.370292
v3-stnn_07-28-15-46-40_2751,87.6979,d,100,10,2,sigmoid,100,0:13:17.372115
v3-stnn_07-28-15-43-10_2483,38.2696,d,10,10,2,sigmoid,100,2:02:40.400952


In [15]:
best_feb_folder = os.path.abspath(os.path.join(os.getcwd(), '..', 'output_result_batch', 'batch_feb'))
feb_result = result.Printer(best_feb_folder)
pd.options.display.max_rows = None
# a = jar_result.get_df(col=['train_loss', 'test_loss', 'mode']).sort_values('test_loss', ascending=True)
feb_df = feb_result.get_df(col=['minrmse','data_normalize', 'batch_size', 'nz', 'nlayers', 'activation', 'nhid', 'time'], increase=False, required_list='classical').sort_values('time')
result.fliter_row(feb_df, "ori")

,minrmse,data_normalize,batch_size,nz,nlayers,activation,nhid,time
ori-stnn_07-28-15-46-44_8127,180.917,d,900,10,4,sigmoid,100,0:05:35.785420
ori-stnn_07-28-15-46-40_2547,156.871,d,700,10,4,sigmoid,100,0:06:30.207248
ori-stnn_07-28-15-46-40_7138,123.322,d,500,10,4,sigmoid,100,0:08:41.120935
ori-stnn_07-28-15-46-40_7349,196.646,d,300,10,4,sigmoid,100,0:09:37.554698
ori-stnn_07-28-15-46-40_7012,148.427,d,100,10,4,sigmoid,100,0:17:48.022802
ori-stnn_07-28-15-43-10_7485,99.2252,d,10,10,4,sigmoid,100,2:20:37.049234


# Batch Mar

In [16]:
best_mar_folder = os.path.abspath(os.path.join(os.getcwd(), '..', 'output_result_batch', 'batch_mar'))
mar_result = result.Printer(best_mar_folder)
pd.options.display.max_rows = None
# a = jar_result.get_df(col=['train_loss', 'test_loss', 'mode']).sort_values('test_loss', ascending=True)
mar_df = mar_result.get_df(col=['minrmse','data_normalize', 'batch_size', 'nz', 'nlayers', 'activation', 'nhid', 'time'], increase=False, required_list='classical').sort_values('time')
result.fliter_row(mar_df, "v3")

,minrmse,data_normalize,batch_size,nz,nlayers,activation,nhid,time
v3-stnn_09-10-10-06-47_8901,8.12984,x,700,40,1,sigmoid,0,0:07:18.842644
v3-stnn_09-10-10-06-43_4899,9.68861,x,500,40,1,sigmoid,0,0:07:44.220410
v3-stnn_09-10-10-06-47_2470,4.15957,x,900,40,1,sigmoid,0,0:12:29.812149
v3-stnn_09-10-10-06-37_8532,3.50459,x,300,40,1,sigmoid,0,0:12:33.137620
v3-stnn_09-10-10-06-37_6236,3.40797,x,100,40,1,sigmoid,0,0:20:00.772245


In [18]:
best_mar_folder = os.path.abspath(os.path.join(os.getcwd(), '..', 'output_result_batch', 'batch_mar'))
mar_result = result.Printer(best_mar_folder)
pd.options.display.max_rows = None
# a = jar_result.get_df(col=['train_loss', 'test_loss', 'mode']).sort_values('test_loss', ascending=True)
mar_df = mar_result.get_df(col=['minrmse','data_normalize', 'batch_size', 'nz', 'nlayers', 'activation', 'nhid', 'time'], increase=False, required_list='classical').sort_values('time')
result.fliter_row(mar_df, "classical")

,minrmse,data_normalize,batch_size,nz,nlayers,activation,nhid,time
classical-stnn_09-10-10-06-43_6189,58.3207,d,500,10,3,sigmoid,50,0:06:16.244762
classical-stnn_09-10-10-06-47_1806,58.3976,d,700,10,3,sigmoid,50,0:07:54.292790
classical-stnn_09-10-10-06-47_3531,58.2968,d,900,10,3,sigmoid,50,0:09:17.147864
classical-stnn_09-10-10-06-37_6908,58.3381,d,300,10,3,sigmoid,50,0:10:54.772481
classical-stnn_09-10-10-06-37_9832,58.359,d,100,10,3,sigmoid,50,0:17:35.176062


In [19]:
best_mar_folder = os.path.abspath(os.path.join(os.getcwd(), '..', 'output_result_batch', 'batch_mar'))
mar_result = result.Printer(best_mar_folder)
pd.options.display.max_rows = None
# a = jar_result.get_df(col=['train_loss', 'test_loss', 'mode']).sort_values('test_loss', ascending=True)
mar_df = mar_result.get_df(col=['minrmse','data_normalize', 'batch_size', 'nz', 'nlayers', 'activation', 'nhid', 'time'], increase=False, required_list='classical').sort_values('time')
result.fliter_row(mar_df, "ori")

,minrmse,data_normalize,batch_size,nz,nlayers,activation,nhid,time
ori-stnn_09-10-10-06-47_8896,54.2187,x,700,40,1,sigmoid,0,0:06:34.325983
ori-stnn_09-10-10-06-43_0210,51.0208,x,500,40,1,sigmoid,0,0:07:14.153523
ori-stnn_09-10-10-06-47_2168,80.4517,x,900,40,1,sigmoid,0,0:10:16.398187
ori-stnn_09-10-10-06-37_9451,4.01588,x,300,40,1,sigmoid,0,0:11:59.022349
ori-stnn_09-10-10-06-37_8489,3.68422,x,100,40,1,sigmoid,0,0:20:49.033419
